original caco2 data set consists of 906 molecule instances with corresponding SMILES representations and their permeability values.

use `rdkit` amd `guache1` modules to acess five extra molecue representations as objectives and other features such as `ecfp_fingerprints` and `fragments` as domain.

In [4]:
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from rdkit.Chem import MolFromSmiles
from rdkit.Chem import  QED
from rdkit.Chem import  rdMolDescriptors
import numpy as np
import pandas as pd
# from gauche.dataloader import DataLoaderMP
from gauche.dataloader import MolPropLoader
from gauche.representations.fingerprints import (
    # ecfp_fingerprints,
    fragments,
    mqn_features,
    )
from gauche.representations.strings import bag_of_characters
from gauche.representations.graphs import molecular_graphs

finger prints

shape: 910 by 2048

In [5]:
original = pd.read_csv('Caco-2.csv')
smiles = original['Drug']

#molecule objects in rdkit
molecules = [MolFromSmiles(s) for s in smiles]

#define Morgen generator
generator = GetMorganGenerator(radius = 3, fpSize = 2048)

#get finger prints
fps = [generator.GetFingerprint(mol) for mol in molecules]
fps_array = np.array(fps)

fragments

shape: 910 by 85

In [6]:
#return np.array 
frag = fragments(smiles= smiles)

mqn feature

910 by 42

In [7]:
mqn = mqn_features(smiles= smiles)

Synthesized Domain

910 by 2175

In [8]:
domain = np.concatenate((fps_array, frag, mqn), axis = 1)

Targets

QED score

In [9]:
qed = np.array([QED.qed(mol) for mol in molecules]).reshape((910,-1))

TPSA

In [10]:
tpsa = np.array([rdMolDescriptors.CalcTPSA(mol) for mol in molecules]).reshape((910,-1))

Permeability

In [11]:
permeability = np.array(original['Y']).reshape((910,-1))

Synthesized Target

910 by 3

In [12]:
target = np.concatenate((qed, tpsa, permeability), axis = 1)
target_names = ['QED', 'TPSA', 'permeability']

In [13]:
np.savetxt('target.csv', target, delimiter= ',',header = ','.join(target_names), fmt='%.8f', comments = '')